In [1]:
# Step 1: Import Required Libraries
import os
import random
import time
import pybullet as p
import pybullet_data
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np


In [2]:
# Step 2: Convert All Images to JPEG Format
folder_path = 'D:\WasteSortingRobo'

# Loop through all files and convert to JPEG
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.bmp', '.tiff', '.jpeg', '.jpg')):
        file_path = os.path.join(folder_path, filename)

      
        with Image.open(file_path) as img:
            img = img.convert("RGB")

            # Save the file in JPEG format
            base_filename = os.path.splitext(filename)[0]
            new_file_path = os.path.join(folder_path, base_filename + ".jpeg")
            img.save(new_file_path, "JPEG")

        print(f"Converted {filename} to JPEG.")

print("All files converted into JPEG.")


All files converted.


In [4]:
'''# Step 3: Clean and Verify the Dataset

# Define allowed image extensions
allowed_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}

# Function to verify and clean the dataset
def filter_and_verify_image(filepath):
    ext = os.path.splitext(filepath)[1].lower()
    
    if ext not in allowed_extensions:
        return False

    try:
        img = Image.open(filepath)
        img.verify()  
        return True
    except (IOError, SyntaxError):
        print(f"Invalid image found: {filepath}")
        return False

# Clean the dataset by removing invalid images
data_dir = 'D:\WasteSortingRobo'
for root, dirs, files in os.walk(data_dir):
    for file in files:
        filepath = os.path.join(root, file)
        if not filter_and_verify_image(filepath):
            print(f"Removing invalid file: {filepath}")
            os.remove(filepath)'''

'# Step 3: Clean and Verify the Dataset\n\n# Define allowed image extensions\nallowed_extensions = {\'.jpg\', \'.jpeg\', \'.png\', \'.bmp\', \'.gif\'}\n\n# Function to verify and clean the dataset\ndef filter_and_verify_image(filepath):\n    ext = os.path.splitext(filepath)[1].lower()\n    \n    if ext not in allowed_extensions:\n        return False\n\n    try:\n        img = Image.open(filepath)\n        img.verify()  \n        return True\n    except (IOError, SyntaxError):\n        print(f"Invalid image found: {filepath}")\n        return False\n\n# Clean the dataset by removing invalid images\ndata_dir = \'D:\\WasteSortingRobo\'\nfor root, dirs, files in os.walk(data_dir):\n    for file in files:\n        filepath = os.path.join(root, file)\n        if not filter_and_verify_image(filepath):\n            print(f"Removing invalid file: {filepath}")\n            os.remove(filepath)'

In [5]:
# Step 4: Build and Train the Convolutional Neural Network (CNN) Model

# Load the dataset using ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

#Outputting how tensorflow interprets the folder names
print(train_generator.class_indices)

# Define a simple CNN model for waste classification
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(64, 64, 3)),  
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30  
)

Found 15 images belonging to 4 classes.
Found 3 images belonging to 4 classes.
{'Plasticbottles': 0, 'Plasticcovers': 1, 'Plasticcups': 2, 'Tincans': 3}
Epoch 1/30
1/1 [==============================] - 2s 2s/step - loss: 1.3863 - accuracy: 0.1333 - val_loss: 1.3775 - val_accuracy: 0.6667
Epoch 2/30
1/1 [==============================] - 0s 222ms/step - loss: 1.3780 - accuracy: 0.6667 - val_loss: 1.3534 - val_accuracy: 0.6667
Epoch 3/30
1/1 [==============================] - 0s 233ms/step - loss: 1.3555 - accuracy: 0.6667 - val_loss: 1.3097 - val_accuracy: 0.6667
Epoch 4/30
1/1 [==============================] - 0s 237ms/step - loss: 1.3148 - accuracy: 0.6667 - val_loss: 1.2379 - val_accuracy: 0.6667
Epoch 5/30
1/1 [==============================] - 0s 203ms/step - loss: 1.2481 - accuracy: 0.6667 - val_loss: 1.1326 - val_accuracy: 0.6667
Epoch 6/30
1/1 [==============================] - 0s 218ms/step - loss: 1.1510 - accuracy: 0.6667 - val_loss: 0.9982 - val_accuracy: 0.6667
Epoch 7/30

In [7]:
# Step 5: PyBullet Simulation for Waste Sorting

# Connect to PyBullet simulation
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.loadURDF("plane.urdf")  
p.setGravity(0, 0, -9.8)  

# Load the robot model (using KUKA iiwa arm as an example)
robot_id = p.loadURDF("kuka_iiwa/model.urdf", useFixedBase=True)

# Define object positions and load objects (e.g., for bottles and cans)
object_id_1 = p.loadURDF("sphere2.urdf", [0, 0, 1], globalScaling=0.5)  
object_id_2 = p.loadURDF("cube.urdf", [1, 0, 1], globalScaling=0.3)  

# Run the simulation loop
for _ in range(10000):
    p.stepSimulation()
    time.sleep(1./240.)

error: Not connected to physics server.

In [ ]:
# Step 6: Clean up PyBullet
p.disconnect()

# Save and load the trained model
model.save('waste_sorting_model.h5')